In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Dense,  Conv2D, MaxPooling2D, TimeDistributed, Flatten, InputLayer, Reshape, Conv1D, MaxPooling1D, Bidirectional, Dropout, ReLU
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adadelta, SGD, Adagrad
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from sklearn.metrics import r2_score
import random
from datetime import datetime, timedelta
import math

#%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#from google.colab import drive
#drive.mount('/content/drive')

random.seed(1)

# LOADING DATASETS
Converting J to KWh and all the date/time strings to timestamps

In [ ]:
#df_2017 = pd.read_csv('/content/drive/MyDrive/eplusout_2017_with_year.csv')[:-1]
df_2017 = pd.read_csv('../data/output_eplus/eplusout_2017_with_year.csv')[:-1]
df_2017['DistrictHeating:Facility [J](TimeStep)'] = df_2017['DistrictHeating:Facility [J](TimeStep)']/3.6e6
df_2017['DistrictCooling:Facility [J](TimeStep)'] = df_2017['DistrictCooling:Facility [J](TimeStep)']/3.6e6
df_2017['InteriorLights:Electricity [J](TimeStep)'] = df_2017['InteriorLights:Electricity [J](TimeStep)']/3.6e6
df_2017['Temp_IN'] = df_2017[['BLOCK1:CORRIDOR:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE2:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE1:Zone Operative Temperature [C](TimeStep)']].mean(1)
df_2017['Heating setpoint'] = 21
df_2017['Heating setpoint'][6*24*134:6*24*257] = 0
df_2017['Cooling setpoint'] = 24
df_2017['Cooling setpoint'][:6*24*104] = 0
df_2017['Cooling setpoint'][6*24*287:] = 0
plt.figure()
plt.title('2017')
df_2017['DistrictHeating:Facility [J](TimeStep)'].plot()
df_2017['DistrictCooling:Facility [J](TimeStep)'].plot()
df_2017['InteriorLights:Electricity [J](TimeStep)'].plot()
plt.show()

#df_2018 = pd.read_csv('/content/drive/MyDrive/eplusout_2018_with_year.csv')[:-1]
df_2018 = pd.read_csv('../data/output_eplus/eplusout_2018_with_year.csv')[:-1]
df_2018['DistrictHeating:Facility [J](TimeStep)'] = df_2018['DistrictHeating:Facility [J](TimeStep)']/3.6e6
df_2018['DistrictCooling:Facility [J](TimeStep)'] = df_2018['DistrictCooling:Facility [J](TimeStep)']/3.6e6
df_2018['InteriorLights:Electricity [J](TimeStep)'] = df_2018['InteriorLights:Electricity [J](TimeStep)']/3.6e6
df_2018['Temp_IN'] = df_2018[['BLOCK1:CORRIDOR:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE2:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE1:Zone Operative Temperature [C](TimeStep)']].mean(1)
df_2018['Heating setpoint'] = 21
df_2018['Heating setpoint'][6*24*134:6*24*257] = 0
df_2018['Cooling setpoint'] = 24
df_2018['Cooling setpoint'][:6*24*104] = 0
df_2018['Cooling setpoint'][6*24*287:] = 0
plt.figure()
plt.title('2018')
df_2018['DistrictHeating:Facility [J](TimeStep)'].plot()
df_2018['DistrictCooling:Facility [J](TimeStep)'].plot()
df_2018['InteriorLights:Electricity [J](TimeStep)'].plot()
plt.show()

#df_2019 = pd.read_csv('/content/drive/MyDrive/eplusout_2019_with_year.csv')[:-1]
df_2019 = pd.read_csv('../data/output_eplus/eplusout_2019_with_year.csv')[:-1]
df_2019['DistrictHeating:Facility [J](TimeStep)'] = df_2019['DistrictHeating:Facility [J](TimeStep)']/3.6e6
df_2019['DistrictCooling:Facility [J](TimeStep)'] = df_2019['DistrictCooling:Facility [J](TimeStep)']/3.6e6
df_2019['InteriorLights:Electricity [J](TimeStep)'] = df_2019['InteriorLights:Electricity [J](TimeStep)']/3.6e6
df_2019['Temp_IN'] = df_2019[['BLOCK1:CORRIDOR:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE2:Zone Operative Temperature [C](TimeStep)',
                              'BLOCK1:ZONE1:Zone Operative Temperature [C](TimeStep)']].mean(1)
df_2019['Heating setpoint'] = 21
df_2019['Heating setpoint'][6*24*134:6*24*257] = 0
df_2019['Cooling setpoint'] = 24
df_2019['Cooling setpoint'][:6*24*104] = 0
df_2019['Cooling setpoint'][6*24*287:] = 0
plt.figure()
plt.title('2019')
df_2019['DistrictHeating:Facility [J](TimeStep)'].plot()
df_2019['DistrictCooling:Facility [J](TimeStep)'].plot()
df_2019['InteriorLights:Electricity [J](TimeStep)'].plot()
plt.show()

ts = []
for elem in df_2017['Date/Time']:
    date, time = elem.split(' ')
    if (time == '24:00:00'):
        time = '00:00:00'
    timestamp = datetime.strptime(f'{date} {time}', '%m/%d/%Y %H:%M:%S')
    if (time == '00:00:00'):
        timestamp += timedelta(days=1)
    ts.append(timestamp)
df_2017['Date/Time'] = pd.to_datetime(ts, format='%m/%d/%Y %H:%M:%S')

ts = []
for elem in df_2018['Date/Time']:
    date, time = elem.split(' ')
    if (time == '24:00:00'):
        time = '00:00:00'
    timestamp = datetime.strptime(f'{date} {time}', '%m/%d/%Y %H:%M:%S')
    if (time == '00:00:00'):
        timestamp += timedelta(days=1)
    ts.append(timestamp)
df_2018['Date/Time'] = pd.to_datetime(ts, format='%m/%d/%Y %H:%M:%S')

ts = []
for elem in df_2019['Date/Time']:
    date, time = elem.split(' ')
    if (time == '24:00:00'):
        time = '00:00:00'
    timestamp = datetime.strptime(f'{date} {time}', '%m/%d/%Y %H:%M:%S')
    if (time == '00:00:00'):
        timestamp += timedelta(days=1)
    ts.append(timestamp)
df_2019['Date/Time'] = pd.to_datetime(ts, format='%m/%d/%Y %H:%M:%S')

## Join 2017 and 2018 in training set and assign test set

In [ ]:
df_train = pd.concat([df_2017, df_2018], axis=0)
df_train.reset_index(drop=True, inplace=True)
df_test = df_2019

# FEATURES AND REGRESSANDS

In [ ]:
regressands = [
            #   'DistrictHeating:Facility [J](TimeStep)',
              'DistrictCooling:Facility [J](TimeStep)',
            #   'InteriorLights:Electricity [J](TimeStep)',
            #   'Temp_IN'
              ]
features = [
            'Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)',
            'Environment:Site Outdoor Air Dewpoint Temperature [C](TimeStep)',
            'Environment:Site Outdoor Air Barometric Pressure [Pa](TimeStep)',
            'Environment:Site Wind Speed [m/s](TimeStep)',
            'Environment:Site Wind Direction [deg](TimeStep)',
            'Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](TimeStep)',
            'Environment:Site Direct Solar Radiation Rate per Area [W/m2](TimeStep)',
            'Environment:Site Solar Azimuth Angle [deg](TimeStep)',
            'Environment:Site Solar Altitude Angle [deg](TimeStep)',
            'Temp_IN',  # comment out if predicting Temp_IN
            #'Heating setpoint',  # comment out unnecessary features depending on the regressand
            'Cooling setpoint'
            ]

# SINGLE RUN OF THE MODEL

##  Train, val and test generation

### Selecting Data


In [ ]:
days = []
months = []
years = []
hours = []
minutes = []
for timestamp in df_train['Date/Time']:
    days.append(timestamp.day)
    months.append(timestamp.month)
    years.append(timestamp.year)
    hours.append(timestamp.hour)
    minutes.append(timestamp.minute)

X_train = pd.DataFrame()
X_train['day'] = days
X_train['month'] = months
X_train['year'] = years
X_train['hour'] = hours
X_train['minute'] = minutes
X_train.index = df_train.index

X_train = X_train.join(df_train[np.concatenate((features, regressands))])
X_train.describe()

### Normalizing

In [ ]:
X_train = X_train.astype('float32')
normalized_X_train=(X_train-X_train.min())/(X_train.max()-X_train.min())
normalized_X_train.dropna(axis=1, inplace=True)
y_train_min, y_train_max = X_train[regressands].min(), X_train[regressands].max()
y_train_mean, y_train_std = X_train[regressands].mean(), X_train[regressands].std()

### From multivariate time serie to supervised problem
We prepare a train set which will associate a set of previous [window] realizations of the features+regressands with the [forseen] following regressands values

In [ ]:
window = 6  # number of past values used to predict the next ones, in this case one hour (6*10min)
forseen = 6  # number of future sample to predict, in this case one hour (6*10min)

In [ ]:
trainX, trainY = [], []
for j in range(window, len(normalized_X_train.index)-forseen, 1):
    input, val = normalized_X_train.values[j-window:j, :], normalized_X_train.values[j:j+forseen,-len(regressands):]  # I know that regressand is in the last column
    trainX.append(input)
    trainY.append(val)

### Separation of the validation set
We took off some random samples from the train set to generate the val set

In [ ]:
#val_size = round(len(trainX)*0.25)
val_size = 6*24*60  # 2 months
valX = []
valY = []
for v in range(val_size):
    rand_i = random.choice(range(len(trainX)))
    valX.append(trainX.pop(rand_i))
    valY.append(trainY.pop(rand_i))

trainX = np.array(trainX)
trainY = np.array(trainY)
valX = np.array(valX)
valY = np.array(valY)
print('trainX shape ', trainX.shape)
print('trainY shape ', trainY.shape)
print('valX shape ', valX.shape)
print('valY shape ', valY.shape)

### Preparing the test set


In [ ]:
days = []
months = []
years = []
hours = []
minutes = []
for timestamp in df_test['Date/Time']:
    days.append(timestamp.day)
    months.append(timestamp.month)
    years.append(timestamp.year)
    hours.append(timestamp.hour)
    minutes.append(timestamp.minute)

X_test = pd.DataFrame()
X_test['day'] = days
X_test['month'] = months
X_test['year'] = years
X_test['hour'] = hours
X_test['minute'] = minutes
X_test.describe()
X_test.index = df_test.index
X_test = X_test.join(df_test[np.concatenate((features, regressands))])

X_test = X_test.astype('float32')
normalized_X_test=(X_test-X_train.min())/(X_train.max()-X_train.min())
#normalized_X_test=(X_test-X_test.min())/(X_test.max()-X_test.min())

normalized_X_test.dropna(axis=1, inplace=True)
#y_test_min, y_test_max = X_test[regressands].min(), X_test[regressands].max()

testX = []
for t in range(window, len(normalized_X_test.index)-forseen, forseen):
    testX.append(normalized_X_test.values[t-window:t, :])
testX = np.array(testX)

## Keras model

In [ ]:
# Model
model_choice = 'cnn_lstm'  # 'single_lstm', 'stacked_lstm', 'lstm_cnn'

if model_choice == 'single_lstm':
    model = Sequential()
    model.add(InputLayer(input_shape=(window, X_train.shape[1])))
    model.add(LSTM(64)) 
    model.add(Dense(forseen*len(regressands)))
    model.add(ReLU())
    model.add(Reshape((forseen, len(regressands))))

if model_choice == 'stacked_lstm':
    model = Sequential()
    model.add(InputLayer(input_shape=(window, X_train.shape[1])))
    model.add(LSTM(64, return_sequences=True))  # in questo modo LSTM si aspetta di ricevere in input i 50 istanti precedenti, ciascuno con 3 valori
    model.add(LSTM(32)) # add return_sequences=True, in the previous layer
    model.add(Dense(forseen*len(regressands)))
    model.add(ReLU())
    model.add(Reshape((forseen, len(regressands))))

if model_choice == 'cnn_lstm':
    model = Sequential()
    model.add(Reshape((window, normalized_X_train.shape[1], 1), input_shape=(window, normalized_X_train.shape[1])))
    model.add(Conv2D(filters=64, kernel_size=(2,1), strides=1, padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(1,1)))
    model.add(Conv2D(filters=64, kernel_size=(2,1), strides=1, padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(1,1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, activation='tanh'))
    model.add(Dense(32))
    model.add(Dense(forseen*len(regressands)))
    model.add(ReLU())
    model.add(Reshape((forseen, len(regressands))))

#lr_schedule = ExponentialDecay(initial_learning_rate=0.2, decay_steps=8000, decay_rate=0.9, staircase=False)  # 0.0053 17/30
#opt = Adagrad(learning_rate=lr_schedule)
model.compile(loss='mse', optimizer='adam', metrics='mean_absolute_error')
print(model.summary())

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=3)
mc = ModelCheckpoint(f'{regressands[0].split(":")[0]}/{model_choice}.h5', monitor='val_loss', mode='min', save_weights_only=True, save_best_only=True)

## Training phase

In [ ]:
try:
    model.load_weights(f'{regressands[0].split(":")[0]}/{model_choice}.h5')
    print('Already trained model found!')
except:
    history = model.fit(x=trainX, y=trainY, batch_size=10, epochs=30, shuffle=True, validation_data=(valX, valY), callbacks=[es, mc])

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Test phase

In [ ]:
model.load_weights(f'{regressands[0].split(":")[0]}/{model_choice}.h5')
final_pred = {}

out = model.predict(testX)

for i, reg in enumerate(regressands):
    temp = out[:, :, i] * (y_train_max[i] - y_train_min[i]) + y_train_min[i]
    # temp = out[:, :, i] * y_train_std[i]
    final_pred[reg] = temp.reshape((testX.shape[0]*forseen))
    err = (abs(X_test.values[window:window+len(final_pred[reg]), -len(regressands)+i]-final_pred[reg])).mean()
    rmse = math.sqrt(((X_test.values[window:window+len(final_pred[reg]), -len(regressands)+i]-final_pred[reg])**2).mean())
    print(f'MAE ({regressands[0].split(":")[0]}): ', round(err, 4))
    print(f'RMSE ({regressands[0].split(":")[0]}): ', round(rmse, 4))

    fig = plt.figure(figsize=(18,10))
    plt.title(reg)
    plt.plot(X_test.values[window:window+len(final_pred[reg]), 5+len(features)+i], label='original')
    plt.plot(final_pred[reg], label=f'prediction')
    plt.ylabel(f'{regressands[0].split(":")[0]} [kWh]')
    plt.xlabel('Time steps (10 minutes resolution)')
    plt.title(f'{regressands[0].split(":")[0]} for the year 2019')
    plt.legend()
    plt.show()



In [ ]:
# July 2019
for i, reg in enumerate(regressands):
    fig = plt.figure(figsize=(18,10))
    plt.title(reg)
    plt.plot(X_test.values[6*24*181:6*24*212, 5+len(features)+i], label='original')
    plt.plot(final_pred[reg][6*24*181:6*24*212], label=f'prediction')
    plt.ylabel(f'{regressands[0].split(":")[0]} [kWh]')
    plt.xlabel('Time steps (10 minutes resolution)')
    plt.title(f'{regressands[0].split(":")[0]} for July 2019')
    plt.legend()
    plt.show()


# (EXTRA) FORECASTING WINDOW OPTIMIZATION

In [ ]:
# Loading the dataset and splitting it into train and test subsets
days = []
months = []
hours = []
years = []
minutes = []
for timestamp in df_train['Date/Time']:
    days.append(timestamp.day)
    months.append(timestamp.month)
    hours.append(timestamp.hour)
    years.append(timestamp.year)
    minutes.append(timestamp.minute)

X_train = pd.DataFrame()
X_train['day'] = days
X_train['month'] = months
X_train['year'] = years
X_train['hour'] = hours
X_train['minute'] = minutes

X_train.index = df_train.index
X_train = X_train.join(df_train[np.concatenate((features, regressands))])

X_train = X_train.astype('float32')
normalized_X_train=(X_train-X_train.min())/(X_train.max()-X_train.min())
y_train_min, y_train_max = X_train[regressands].min(), X_train[regressands].max()


# Loading the test dataset
days = []
months = []
years = []
hours = []
minutes = []
for timestamp in df_test['Date/Time']:
    days.append(timestamp.day)
    months.append(timestamp.month)
    years.append(timestamp.year)
    hours.append(timestamp.hour)
    minutes.append(timestamp.minute)

X_test = pd.DataFrame()
X_test['day'] = days
X_test['month'] = months
X_test['year'] = years
X_test['hour'] = hours
X_test['minute'] = minutes
X_test.index = df_test.index
X_test = X_test.join(df_test[np.concatenate((features, regressands))])

X_test = X_test.astype('float32')
normalized_X_test=(X_test-X_train.min())/(X_train.max()-X_train.min())
#normalized_X_test=(X_test-X_test.min())/(X_test.max()-X_test.min())
y_test_min, y_test_max = X_test[regressands].min(), X_test[regressands].max()

In [ ]:
sizes = np.array([1, 3, 6, 12, 24, 48, 72, 96, 120, 144, 168])*6
mae_values = {}
#pred_values = {}
#real_values = {}
rmse_values = {}
for i in sizes:
    window = i
    forseen = i

    trainX, trainY = [], []
    for j in range(window, len(normalized_X_train.index)-forseen, 1):
        input, val = normalized_X_train.values[j-window:j, :], normalized_X_train.values[j:j+forseen,-len(regressands):]  # I know that regressands are the last columns
        trainX.append(input)
        trainY.append(val)

    #val_size = round(len(trainX)*0.25)
    val_size = 6*24*60  # 2 months
    valX = []
    valY = []
    for v in range(val_size):
        rand_i = random.choice(range(len(trainX)))
        valX.append(trainX.pop(rand_i))
        valY.append(trainY.pop(rand_i))

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    valX = np.array(valX)
    valY = np.array(valY)

    testX = []
    for t in range(window, len(normalized_X_test.index)-forseen, forseen):
        testX.append(normalized_X_test.values[t-window:t, :])
    testX = np.array(testX)

    # Model
    model = Sequential()
    model.add(Reshape((window, X_train.shape[1], 1), input_shape=(window, X_train.shape[1])))
    model.add(Conv2D(filters=64, kernel_size=(2,1), strides=1, padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(1,1)))
    model.add(Conv2D(filters=64, kernel_size=(2,1), strides=1, padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,1), strides=(1,1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, activation='tanh'))
    model.add(Dense(32))
    model.add(Dense(forseen*len(regressands)))
    model.add(ReLU())
    model.add(Reshape((forseen, len(regressands))))

    #lr_schedule = ExponentialDecay(initial_learning_rate=0.2, decay_steps=1000, decay_rate=0.9, staircase=False)  # 0.0053 17/30
    #opt = Adagrad(learning_rate=lr_schedule)
    model.compile(loss='mse', optimizer='adam')

    es = EarlyStopping(monitor='val_loss', mode='min', patience=3)
    mc = ModelCheckpoint(f'{regressands[0].split(":")[0]}/{i//6}H_forecast.h5', monitor='val_loss', mode='min', save_weights_only=True, save_best_only=True)

    print(f'Forecasting {i//6}H ahead...')
    try:
        model.load_weights(f'{regressands[0].split(":")[0]}/{i//6}H_forecast.h5')
        print('Already trained model found!')
    except:
        history = model.fit(x=trainX, y=trainY, batch_size=10, epochs=30, shuffle=True, validation_data=(valX, valY), callbacks=[es, mc])
    print('\n------------------------------\n')

    model.load_weights(f'{regressands[0].split(":")[0]}/{i//6}H_forecast.h5')

    final_pred = model.predict(testX)[:,:,0]*(y_train_max.iloc[0]-y_train_min.iloc[0])+y_train_min.iloc[0]

    final_pred = final_pred.reshape((testX.shape[0]*forseen))

    mae = (abs(X_test.values[window:window+len(final_pred), -1]-final_pred)).mean()
    rmse = math.sqrt(((X_test.values[window:window+len(final_pred), -1]-final_pred)**2).mean())
    
    mae_values[f'{i//6}H'] = [mae]
    rmse_values[f'{i//6}H'] = [rmse]
    #pred_values[f'{i//6}H'] = final_pred[:(len(X_test)-window)]
    #real_values[f'{i//6}H'] = X_test.values[window:, -1]

## Plots

In [ ]:
pred_x = [x//6 for x in sizes]
pred_y_rmse = []
pred_y_mae = []

for k in mae_values.keys():
    pred_y_rmse.append(rmse_values[k][0])
    pred_y_mae.append(mae_values[k][0])

errors_df = pd.DataFrame({'Window [hours]': pred_x, 'MAE': pred_y_mae, 'RMSE': pred_y_rmse})
errors_df.to_csv(f'{regressands[0].split(":")[0]}/errors.csv', index=False)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(pred_x, pred_y_mae, label='MAE')
plt.plot(pred_x, pred_y_rmse, label='RMSE')
plt.xlabel('Forecast window [hours]')
plt.title(f'Error on the predicted {regressands[0].split(":")[0]} [kWh]')
plt.xticks(pred_x)
plt.grid()
plt.savefig(f'{regressands[0].split(":")[0]}/errors_plot')
plt.legend()